## Classification Model

### ECG Heartbeat Classification using 1D CNN

#### Preprocessing

In [3]:
import pandas as pd

In [4]:
train = pd.read_csv("mitbih_train.csv")
test = pd.read_csv("mitbih_test.csv")

In [5]:
X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values

X_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values

#### Normalization

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Label encoding

In [7]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)

#### Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(187, 1)),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\aayus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Before training, reshaping the input

In [9]:
X_train_cnn = X_train.reshape(-1, 187, 1)
X_test_cnn = X_test.reshape(-1, 187, 1)

model.fit(X_train_cnn, y_train_cat, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9872 - loss: 0.0576 - val_accuracy: 0.1387 - val_loss: 40.7268
Epoch 2/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 1.6460e-05 - val_accuracy: 0.1387 - val_loss: 48.0243
Epoch 3/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 1.9451e-06 - val_accuracy: 0.1387 - val_loss: 55.4390
Epoch 4/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 1.6732e-05 - val_accuracy: 0.1387 - val_loss: 62.8495
Epoch 5/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 3.1445e-06 - val_accuracy: 0.1387 - val_loss: 69.6430
Epoch 6/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 5.5465e-07 - val_accuracy: 0.1387 - val_loss: 76.3458
Epoch 7/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 1.0401e-07 - val_accuracy: 0.1387 - val_loss: 80.5591
Epoch 8/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - 

## Evaluation

In [10]:
loss, acc = model.evaluate(X_test_cnn, y_test_cat)
print("Test Accuracy:", acc)

685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9836 - loss: 1.8201
Test Accuracy: 0.827600359916687


#### Confusion Matrix

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = model.predict(X_test_cnn)
y_pred_labels = np.argmax(y_pred, axis=1)

print(confusion_matrix(y_test, y_pred_labels))
print(classification_report(y_test, y_pred_labels))

685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  
[[18117     0     0     0     0]
 [  556     0     0     0     0]
 [ 1448     0     0     0     0]
 [  162     0     0     0     0]
 [ 1608     0     0     0     0]]
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91     18117
         1.0       0.00      0.00      0.00       556
         2.0       0.00      0.00      0.00      1448
         3.0       0.00      0.00      0.00       162
         4.0       0.00      0.00      0.00      1608

    accuracy                           0.83     21891
   macro avg       0.17      0.20      0.18     21891
weighted avg       0.68      0.83      0.75     21891



c:\Users\aayus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aayus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aayus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

#### RandomForest

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.974235987392079


In [13]:
import joblib

In [14]:
joblib.dump(clf, "ecg_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']